In [1]:
test_id = "pic_test"
id_file = open("./ID.txt","w")
print(test_id, file=id_file)
id_file.close()

In [2]:
'''
=====================================
        Hyperparameters
=====================================
'''

from Hyperparameters import Random_Seed

Test_times = 1000  
# NOTE: Total test time is 10*Test_times
# We will output the average influence of 10*'Test_times' tests as long as the std.

In [3]:

'''
=====================================
        Dataset Loading
=====================================
'''

import numpy as np
import networkx as nx

node_feature = np.loadtxt(open("./dataset_tbu/node_features.txt"),dtype=int,delimiter=" ",skiprows=0)
edges = np.loadtxt(open("./dataset_tbu/edges.txt"),dtype=int,delimiter=" ",skiprows=0)

num_nodes = len(node_feature)
num_features = len(node_feature[0])

adjacency = nx.to_dict_of_lists(nx.from_edgelist(edges))
for i in range(num_nodes):
    if not (i in adjacency):
        adjacency.update({i:[]})

In [4]:
# To generate diffusion chians
import diffusion_chains_gen


test_id: pic_test [END]
General_IC


In [5]:

'''
=========================================
        Diffusion Model Loading
=========================================
'''

from Hyperparameters import Scalar_factor, Offset, Diff_Model

import pandas as pd
df = pd.read_csv('./diffusion_model_v.csv')
v = np.array(df.values.tolist()).reshape(-1)
df = pd.read_csv('./diffusion_model_W.csv')
W = np.array(df.values.tolist())


In [6]:
'''
=====================================
            Time Record.
=====================================
'''

import time
subtime = time.perf_counter()


'''
===============================================
        Preparation for Generalized IC
===============================================
'''

def sigmoid(x):
    return 1./ ( 1. + np.exp(-x) )

def softmax(x):
    x -= np.max(x, keepdims = True) 
    x = np.exp(x) / np.sum(np.exp(x), keepdims = True)
    return x

inter_ic_file = open("./IC.txt","w")
inter_lt_file = open("./LT.txt","w")

import random
random.seed(Random_Seed)

probabilities = []
for u in range(num_nodes):
    x = np.ones(len(adjacency[u]))*1./len(adjacency[u])
    for i in range(len(adjacency[u])):
        print("{} {} {:.8f}".format( u, adjacency[u][i], x[i]), file=inter_ic_file)
        
    probabilities = probabilities + [x]

tmp = np.transpose(edges)
tmp = np.transpose(np.array([tmp[1],tmp[0]]))
adjacency_inv = nx.to_dict_of_lists(nx.from_edgelist(tmp))
for i in range(num_nodes):
    if not (i in adjacency_inv):
        adjacency_inv.update({i:[]})
    
for u in range(num_nodes):
    x = np.ones(len(adjacency_inv[u]))*1./len(adjacency_inv[u])
    rnd = random.random()
    for i in range(len(adjacency_inv[u])):
        print("{} {} {:.8f}".format( adjacency_inv[u][i], u, x[i]*rnd), file=inter_lt_file)

if (Diff_Model=="General_IC"):
    '''
    if (os.path.exists("./tmp/known_PIC_input.txt")):
        os.remove("./tmp/known_PIC_input.txt")
    '''
    print("hey")
    inter_pic_file = open("./PIC.txt","w")
    probabilities = []
    for u in range(num_nodes):
        h_u = node_feature[u]
        x = np.empty(len(adjacency[u]))
            
        for i in range(len(adjacency[u])):
            h_w = node_feature[adjacency[u][i]]
            x[i] = v.transpose() @ np.tanh(W @ np.append(h_u,h_w))
                
        x = sigmoid(Scalar_factor * x + Offset).tolist()
        
        for i in range(len(adjacency[u])):
            print("{} {} {:.8f}".format( u, adjacency[u][i], x[i]), file=inter_pic_file)
        
        probabilities = probabilities + [x] 
        
else: 
    pass

    
# Calculate p(u->w), w\in neighborhood(u)

time_IC = time.perf_counter()-subtime

hey


In [7]:

'''
===============================================
            Influence Evaluation
===============================================
'''

import queue
import random
random.seed(Random_Seed)

def diff(seeds):
    random.seed(random.randint(-2147483648,2147483648))
    active = queue.Queue()
    visited = np.zeros(num_nodes,dtype=bool)
    
    for seed in seeds:
        active.put_nowait(seed)
        visited[seed] = True
    
    influence = len(seeds)
    while (not active.empty()):
        u = active.get_nowait()
        x = probabilities[u]
        
        for i in range(len(adjacency[u])):
            r = random.uniform(0,1)
            if ((not visited[adjacency[u][i]]) and r<x[i]):
                node = adjacency[u][i]
                active.put_nowait(node)
                visited[node] = True            # Activate node.
                influence = influence+1
    
    return influence



import comp_algos.IMM.IMM as imm
import comp_algos.IMM_LT.IMM_LT as imm_lt
g = imm_lt.readGraph_direct("./LT.txt")

def est(graph, seeds, R):
    """ 
    Estimate the influence of a seed set with generated RR-sets, R.
    Return fraction of RR-sets covered by node set seeds.
    """
    counter = 0
    for RR in R:
        if len(seeds & RR) != 0:
            counter += 1
    influence = graph.n * counter / len(R)
    return influence

def Sampled_est(seeds):
    
    seeds = set(seeds)
    R = imm.RRCollection(g, int(num_nodes/50))
    influence = est(g, seeds, R)
    
    return influence

def Sampled_est_LT(seeds):
    seeds = set(seeds)
    R = imm_lt.RRCollection(g, int(num_nodes/50))
    influence = est(g, seeds, R)
    
    return influence

    
def eval_(seeds, mode="All"):
    influence = 0
    if (Diff_Model=="LT"):
        for i in range(int(Test_times)):
            influence += Sampled_est_LT(seeds)
    elif mode is "All":
        for i in range(int(Test_times)):
            influence += diff(seeds)      
    elif mode is "Sampled":
        for i in range(int(Test_times)):
            influence += Sampled_est(seeds)
                          
    influence /= int(Test_times)
    return influence


def evaluation(seeds, mode="All"):
    influence = []
    for i in range(10):
        influence += [eval_(seeds,mode)]
        print(i, " ", influence[i], end="\t")
    print()
    influence = np.array(influence)
    return influence.mean(), influence.std()

In [8]:
from DSCOM_pred import seed_select, ablationexperiment

[0.         0.         0.         0.         0.         1.
 0.         1.         0.         0.33141762 0.00192675]


In [9]:

'''
=====================================
        Dataset Loading
=====================================
'''

import torch

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch
from torch_geometric.data import Data


# append "degree" and "1/degree" into the feature
import networkx as nx
deg = np.ndarray((num_nodes,2))
for node in range(num_nodes):
    if (len(adjacency[node])==0):
        deg[node][0] = 0
        deg[node][1] = 0
    else:
        deg[node][0] = len(adjacency[node])
        deg[node][1] = 1./len(adjacency[node])
node_feature = np.concatenate((node_feature,deg),axis=1)

num_features = len(node_feature[0])
feature_max = np.max(node_feature,axis=0)
feature_min = np.min(node_feature,axis=0)
node_feature = node_feature-feature_min
std = feature_max-feature_min
for i in range(len(std)):
    if (std[i]==0):
        std[i] = 1.
node_feature = node_feature/std


import math
for i in node_feature:
    for j in i:
        if (math.isnan(j)):
            print("a")

node_feature = torch.tensor(node_feature).float()
edge = torch.tensor(edges, dtype=torch.long)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = Data(x=node_feature, edge_index = np.transpose(edge)).to(device)



In [10]:

import torch


import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import GATConv

from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree


'''
==============================================
       Definition of DSCOM (GAT part).
==============================================
'''

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# GAT Definition
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hidden1 = 8
        self.in_head = 8
        self.out_head = 1
        self.hidden2 = 10
        
        self.conv1 = GATConv(num_features, self.hidden1, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hidden1*self.in_head, self.hidden2, concat=False, heads=self.out_head, dropout=0.6)
        self.mlp = nn.Linear(self.hidden2*self.out_head, Out_num_features)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x, alpha = self.conv1(x, edge_index, return_attention_weights=True)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.mlp(x)
        x = F.elu(x)
        
        return x, alpha  #F.softmax(x, dim=1), alpha


import time
dscom_time = time.time()
    
import DSCOM_main

dscom_time = time.time() - dscom_time

model = torch.load("./model.pth")


0
tensor(6044.2202, device='cuda:0', grad_fn=<AddBackward0>)
tensor([[-0.0201, -0.0824,  0.1113,  0.1078, -0.2185,  0.0097],
        [ 0.0703, -0.0592,  0.1730,  0.1877, -0.1165, -0.0838],
        [-0.0201, -0.0824,  0.1113,  0.1078, -0.2185,  0.0097],
        ...,
        [ 0.5418, -0.5579, -0.2622,  0.1534, -0.4189,  0.5310],
        [-0.3655, -0.6309,  0.3667, -0.4512, -0.0547, -0.1773],
        [-0.5753,  0.0253,  0.3347, -0.1349, -0.5153, -0.4779]],
       device='cuda:0', grad_fn=<EluBackward0>)
100
tensor(3093.3564, device='cuda:0', grad_fn=<AddBackward0>)
tensor([[ 1.4164e+01, -9.8129e-01,  3.9117e+00,  2.3376e-01,  2.0905e+01,
         -9.9214e-01],
        [ 4.3076e+00, -9.8009e-01,  2.3902e+00, -7.8166e-01,  1.1431e+01,
         -9.6613e-01],
        [ 1.7462e+01, -9.7281e-01,  3.6974e+00, -5.2840e-01,  2.3532e+01,
         -9.9891e-01],
        ...,
        [ 1.5726e+00, -7.2640e-01,  2.8310e-01, -7.6618e-02,  3.5574e+00,
         -7.3290e-01],
        [ 5.4970e-01, -3.0491

## Select 20 Seeds

In [11]:
Num_Seeds = 20

In [12]:

from DSCOM_pred import seed_select
Seeds_mxdg, time_mxdg, Seeds_mxct, time_mxct, Seeds_core, time_core, Seeds_pgRk, time_pgRk = seed_select(model, data, Num_Seeds)

In [13]:

graph = imm.readGraph_direct("./PIC.txt")
    
# Change l to achieve the 1-n^l probability
import math
l = 3 * (1 + math.log(2) / math.log(num_nodes))
EPSILON = 0.5

# Sampling RR-sets
R = imm.Sampling(graph, Num_Seeds, EPSILON, l)
print("Sampling Done.")
# Greedy algorithm
Seeds_IMM_PIC = imm.GreedyCoverage(R, Num_Seeds)
time_IMM_PIC = time_IC+time.perf_counter()-subtime

inf_IMM_PIC = evaluation(Seeds_IMM_PIC)
print(inf_IMM_PIC)


Sampling Done.
0   498.195	1   496.42	2   499.041	3   497.413	4   498.263	5   497.627	6   498.017	7   495.536	8   497.214	9   497.339	
(497.50649999999996, 0.9413741286013708)


In [14]:
inf_IMM_PIC, time_IMM_PIC

((497.50649999999996, 0.9413741286013708), 982.5431299)

In [15]:
'''
# CELF with known Diffusion Model Evaluation
# The best in theory. (Also, impossible to be reached by CELF in reality since the diffusion model is unknown in reality.)

df = pd.read_csv('./Seeds_Selection/CELF_diffusion_known.csv')
Seeds_CELF_known = np.array(df.values.tolist()).reshape(-1)
inf_CELF = evaluation(Seeds_CELF_known)
'''

'''
print("Method\tinfluence\tSeeds\ttime\n")

subtime = time.perf_counter()

import comp_algos.IMM.IMM as imm
import comp_algos.IMM_LT.IMM_LT as imm_lt
import math

seeds = [5,10,15,20,25,30,35,40,45,50,100]
for seed in seeds:
    # IMM-PIC
    # Read in graph
    print(seed)
    graph = imm.readGraph_direct("./log/"+test_id+"/PIC.txt")
    
    # Change l to achieve the 1-n^l probability
    l = 3 * (1 + math.log(2) / math.log(num_nodes))
    EPSILON = 0.5
    
    # Sampling RR-sets
    R = imm.Sampling(graph, seed, EPSILON, l)
    print("oh")
    # Greedy algorithm
    Seeds_IMM_PIC = imm.GreedyCoverage(R, seed)
    time_IMM_PIC = time_IC+time.perf_counter()-subtime

    inf_IMM_PIC = evaluation(Seeds_IMM_PIC)
    print(inf_IMM_PIC)
'''

pass

In [16]:
'''
# CELF with known Diffusion Model Evaluation
# The best in theory. (Also, impossible to be reached by CELF in reality since the diffusion model is unknown in reality.)

df = pd.read_csv('./Seeds_Selection/CELF_diffusion_known.csv')
Seeds_CELF_known = np.array(df.values.tolist()).reshape(-1)
inf_CELF = evaluation(Seeds_CELF_known)
'''

'''
# IMM-IC (IC Model Only!)
subtime = time.perf_counter()

import comp_algos.IMM.IMM as imm
import comp_algos.IMM_LT.IMM_LT as imm_lt
import math
# Read in graph
graph = imm.readGraph_direct("./IC.txt")
    
# Change l to achieve the 1-n^l probability
l = 3 * (1 + math.log(2) / math.log(num_nodes))
EPSILON = 0.5
    
# Sampling RR-sets
Num_Seeds = 20
R = imm.Sampling(graph, min(Num_Seeds,170), EPSILON, l)
    
# Greedy algorithm
Seeds_IMM_IC = imm.GreedyCoverage(R, Num_Seeds)
time_IMM_IC = time_IC+time.perf_counter()-subtime

inf_IMM_IC = evaluation(Seeds_IMM_IC)
'''

pass

In [17]:
# inf_IMM_IC, time_IMM_IC

In [18]:
'''
# IMM-LT (LT Model only!)
subtime = time.perf_counter()

Num_Seeds = 5

l = 3 * (1 + math.log(2) / math.log(num_nodes))
EPSILON = 0.5

graph = imm_lt.readGraph_direct("./LT.txt")
R = imm_lt.Sampling(graph, min(Num_Seeds,100), EPSILON, l)
Seeds_IMM_LT = imm_lt.GreedyCoverage(R, Num_Seeds)

print("DONE")

time_IMM_LT = time_IC+time.perf_counter()-subtime
inf_IMM_LT = evaluation(Seeds_IMM_LT)
'''

pass

In [19]:
# inf_IMM_LT, time_IMM_LT

In [20]:

inf_dscom_mxdg = evaluation(Seeds_mxdg)
inf_dscom_mxct = evaluation(Seeds_mxct)
inf_dscom_core = evaluation(Seeds_core)
inf_dscom_pgRk = evaluation(Seeds_pgRk)


0   459.199	1   457.033	2   452.543	3   459.793	4   452.965	5   459.323	6   451.864	7   458.191	8   451.717	9   451.741	
0   457.475	1   460.732	2   453.831	3   455.985	4   455.641	5   457.485	6   455.039	7   456.142	8   460.545	9   455.196	
0   451.403	1   451.783	2   453.381	3   458.554	4   452.927	5   449.968	6   451.294	7   451.385	8   451.971	9   451.845	
0   453.212	1   459.992	2   456.895	3   454.529	4   461.712	5   458.602	6   452.223	7   457.616	8   458.322	9   458.93	


In [26]:

np.random.seed(Random_Seed)
Seeds_Rand = np.random.randint(0,num_nodes-1,size=Num_Seeds)
inf_Rand = evaluation(Seeds_Rand)

0   161.322	1   161.29	2   163.806	3   172.982	4   162.114	5   154.869	6   166.878	7   164.694	8   160.102	9   170.424	


In [31]:
from Hyperparameters import Num_Chains, Window_length_half, Scalar_factor, Offset

print('Num_Chains = ',Num_Chains)
print('Window_length_half = ',Window_length_half)   
if (Diff_Model=="General_IC"):
    print("Generalized IC: sigmoid(a*score(h_u,h_w)+b)")
    print("a = ",Scalar_factor)
    print("b = ",Offset)
elif (Diff_Model=="IC"):
    print("IC")
print("Num_Seeds = ",Num_Seeds)
print()
print("Method\tinfluence\tSeeds\ttime\n")
print()
print("\n==================================\n\t\tDSCOM\n==================================")
print()
# print("DSCOM Trainging time:\t",dscom_time)
print("DSCOM + max-degree:\t",inf_dscom_mxdg,"\t",Seeds_mxdg,"\t",time_mxdg)
print("DSCOM + k-core:\t",inf_dscom_core,"\t",Seeds_core,"\t",time_core)
print("DSCOM + max-closeness:\t",inf_dscom_mxct,"\t",Seeds_mxct,"\t",time_mxct)
print("DSCOM + pageRank:\t",inf_dscom_pgRk,"\t",Seeds_pgRk,"\t",time_pgRk)
print("\n")
print("\n===============================================\n\t\t Random \n===============================================")
print(inf_Rand,"\t",Seeds_Rand,"\t")
print("\n===============================================\n\tIMM (given correct diffusion model)\n===============================================")
print(inf_IMM_PIC,"\t",Seeds_IMM_PIC,"\t",time_IMM_PIC)

Num_Chains =  500
Window_length_half =  2
Generalized IC: sigmoid(a*score(h_u,h_w)+b)
a =  0.5
b =  -6
Num_Seeds =  20

Method	influence	Seeds	time



		DSCOM

DSCOM + max-degree:	 (455.4369, 3.3632019401160016) 	 [0, 107, 1684, 3980, 856, 348, 686, 3437, 1912, 806, 870, 414, 776, 583, 1733, 896, 56, 483, 346, 3596] 	 78.93424649999986
DSCOM + k-core:	 (452.45110000000005, 2.2163154761901454) 	 [0, 107, 1684, 3980, 729, 348, 686, 3437, 1912, 689, 858, 414, 776, 577, 901, 896, 9, 353, 24, 3448] 	 79.26001959999962
DSCOM + max-closeness:	 (456.8071, 2.175761082931676) 	 [0, 107, 1684, 3980, 856, 348, 686, 3437, 1912, 806, 870, 414, 776, 583, 1733, 917, 56, 483, 346, 1085] 	 83.84794729999976
DSCOM + pageRank:	 (457.2033, 2.871335614309131) 	 [0, 107, 1684, 3980, 856, 348, 686, 3437, 1912, 833, 870, 414, 776, 583, 1837, 917, 25, 483, 346, 1085] 	 79.16271399999982



		 Random 
(163.8481, 4.971497409231951) 	 [ 733  700  311 1124   89 1095 3926  128 2824   53 2371 2310 3597  316
  636 334

## Select 200 Seeds

In [32]:
Num_Seeds = 200

In [33]:

from DSCOM_pred import seed_select
Seeds_mxdg, time_mxdg, Seeds_mxct, time_mxct, Seeds_core, time_core, Seeds_pgRk, time_pgRk = seed_select(model, data, Num_Seeds)

In [34]:

graph = imm.readGraph_direct("./PIC.txt")
    
# Change l to achieve the 1-n^l probability
import math
l = 3 * (1 + math.log(2) / math.log(num_nodes))
EPSILON = 0.5

# Sampling RR-sets
R = imm.Sampling(graph, min(Num_Seeds,170), EPSILON, l)
print("Sampling Done.")
# Greedy algorithm
Seeds_IMM_PIC = imm.GreedyCoverage(R, Num_Seeds)
time_IMM_PIC = time_IC+time.perf_counter()-subtime

inf_IMM_PIC = evaluation(Seeds_IMM_PIC)
print(inf_IMM_PIC)

Sampling Done.
0   818.903	1   814.632	2   817.55	3   818.125	4   816.461	5   817.318	6   817.095	7   817.725	8   819.294	9   815.966	
(817.3069, 1.3081654673626046)


In [35]:
inf_IMM_PIC, time_IMM_PIC

((817.3069, 1.3081654673626046), 3322.6727473)

In [36]:
'''
# CELF with known Diffusion Model Evaluation
# The best in theory. (Also, impossible to be reached by CELF in reality since the diffusion model is unknown in reality.)

df = pd.read_csv('./Seeds_Selection/CELF_diffusion_known.csv')
Seeds_CELF_known = np.array(df.values.tolist()).reshape(-1)
inf_CELF = evaluation(Seeds_CELF_known)
'''

'''
print("Method\tinfluence\tSeeds\ttime\n")

subtime = time.perf_counter()

import comp_algos.IMM.IMM as imm
import comp_algos.IMM_LT.IMM_LT as imm_lt
import math

seeds = [5,10,15,20,25,30,35,40,45,50,100]
for seed in seeds:
    # IMM-PIC
    # Read in graph
    print(seed)
    graph = imm.readGraph_direct("./log/"+test_id+"/PIC.txt")
    
    # Change l to achieve the 1-n^l probability
    l = 3 * (1 + math.log(2) / math.log(num_nodes))
    EPSILON = 0.5
    
    # Sampling RR-sets
    R = imm.Sampling(graph, seed, EPSILON, l)
    print("oh")
    # Greedy algorithm
    Seeds_IMM_PIC = imm.GreedyCoverage(R, seed)
    time_IMM_PIC = time_IC+time.perf_counter()-subtime

    inf_IMM_PIC = evaluation(Seeds_IMM_PIC)
    print(inf_IMM_PIC)
'''

pass

In [37]:
'''
# CELF with known Diffusion Model Evaluation
# The best in theory. (Also, impossible to be reached by CELF in reality since the diffusion model is unknown in reality.)

df = pd.read_csv('./Seeds_Selection/CELF_diffusion_known.csv')
Seeds_CELF_known = np.array(df.values.tolist()).reshape(-1)
inf_CELF = evaluation(Seeds_CELF_known)
'''

'''
# IMM-IC (IC Model Only!)
subtime = time.perf_counter()

import comp_algos.IMM.IMM as imm
import comp_algos.IMM_LT.IMM_LT as imm_lt
import math
# Read in graph
graph = imm.readGraph_direct("./IC.txt")
    
# Change l to achieve the 1-n^l probability
l = 3 * (1 + math.log(2) / math.log(num_nodes))
EPSILON = 0.5
    
# Sampling RR-sets
R = imm.Sampling(graph, min(Num_Seeds,170), EPSILON, l)
    
# Greedy algorithm
Seeds_IMM_IC = imm.GreedyCoverage(R, Num_Seeds)
time_IMM_IC = time_IC+time.perf_counter()-subtime

inf_IMM_IC = evaluation(Seeds_IMM_IC)
'''

pass

In [38]:
# inf_IMM_IC, time_IMM_IC

In [39]:

'''
# IMM-LT (LT Model only!)
subtime = time.perf_counter()

Num_Seeds = 200

graph = imm_lt.readGraph_direct("./LT.txt")
R = imm_lt.Sampling(graph, min(Num_Seeds,100), EPSILON, l)
Seeds_IMM_LT = imm_lt.GreedyCoverage(R, Num_Seeds)

time_IMM_LT = time_IC+time.perf_counter()-subtime
inf_IMM_LT = evaluation(Seeds_IMM_LT)
'''
pass

In [40]:
# inf_IMM_LT, time_IMM_LT

In [41]:

inf_dscom_mxdg = evaluation(Seeds_mxdg)
inf_dscom_mxct = evaluation(Seeds_mxct)
inf_dscom_core = evaluation(Seeds_core)
inf_dscom_pgRk = evaluation(Seeds_pgRk)


0   741.476	1   736.364	2   739.049	3   737.665	4   736.032	5   739.567	6   742.332	7   739.731	8   738.295	9   740.728	
0   737.295	1   736.86	2   737.675	3   736.605	4   735.342	5   737.152	6   736.145	7   736.744	8   736.539	9   734.794	
0   709.503	1   712.95	2   712.604	3   708.658	4   714.132	5   712.884	6   712.505	7   712.208	8   711.079	9   712.285	
0   736.988	1   739.726	2   739.425	3   736.518	4   740.838	5   741.002	6   741.309	7   738.357	8   739.663	9   739.845	


In [45]:
from Hyperparameters import Num_Chains, Window_length_half, Scalar_factor, Offset

print('Num_Chains = ',Num_Chains)
print('Window_length_half = ',Window_length_half)   
if (Diff_Model=="General_IC"):
    print("Generalized IC: sigmoid(a*score(h_u,h_w)+b)")
    print("a = ",Scalar_factor)
    print("b = ",Offset)
elif (Diff_Model=="IC"):
    print("IC")
print("Num_Seeds = ",Num_Seeds)
print()
print("Method\tinfluence\tSeeds\ttime\n")
print()
print("\n==================================\n\t\tDSCOM\n==================================")
print()
# print("DSCOM Trainging time:\t",dscom_time)
print("DSCOM + max-degree:\t",inf_dscom_mxdg,"\t",Seeds_mxdg,"\t",time_mxdg)
print("DSCOM + k-core:\t",inf_dscom_core,"\t",Seeds_core,"\t",time_core)
print("DSCOM + max-closeness:\t",inf_dscom_mxct,"\t",Seeds_mxct,"\t",time_mxct)
print("DSCOM + pageRank:\t",inf_dscom_pgRk,"\t",Seeds_pgRk,"\t",time_pgRk)
print("\n")
print("\n===============================================\n\tIMM (given correct diffusion model)\n===============================================")
print(inf_IMM_PIC,"\t",Seeds_IMM_PIC,"\t",time_IMM_PIC)

Num_Chains =  500
Window_length_half =  2
Generalized IC: sigmoid(a*score(h_u,h_w)+b)
a =  0.5
b =  -6
Num_Seeds =  200

Method	influence	Seeds	time



		DSCOM

DSCOM + max-degree:	 (739.1239, 1.9738361862120055) 	 [3101, 3717, 859, 71, 107, 785, 757, 3502, 182, 33, 3996, 717, 453, 480, 2281, 3987, 179, 1002, 244, 356, 3548, 2691, 1055, 979, 3990, 377, 3861, 386, 89, 81, 864, 2266, 3546, 2650, 1352, 583, 2171, 8, 3999, 220, 2047, 4000, 604, 3538, 174, 2111, 3477, 49, 3500, 175, 3437, 424, 3652, 910, 3487, 776, 3774, 2670, 2861, 1240, 3616, 1202, 2732, 2886, 2671, 312, 3479, 4, 3327, 3720, 2774, 573, 7, 789, 3061, 173, 2983, 594, 23, 1043, 3057, 206, 4020, 490, 198, 1174, 1097, 3945, 127, 437, 3455, 4023, 137, 1733, 2017, 870, 2313, 3505, 3985, 2696, 171, 2438, 435, 77, 857, 414, 2049, 1322, 2809, 432, 3363, 2978, 729, 2730, 2926, 3662, 3617, 429, 3592, 423, 146, 73, 708, 584, 478, 398, 2713, 961, 3825, 713, 346, 3981, 787, 212, 937, 3, 2668, 1032, 720, 355, 1831, 2126, 1444, 1041, 3948